In [21]:
#initial

import pandas as pd
import random
import time
import matplotlib.pyplot as plt
from datetime import datetime

words = pd.read_excel('dutch.xlsx',sheet_name='update')
words = words.loc[:,'word':]
lesson = pd.read_excel('dutch.xlsx',sheet_name='lesson')
lesson = lesson.loc[:,'lesson':]

In [23]:
import math

In [59]:
#classes

class Words(object):
    def __init__(self, word, typ, translation, russian, example, example_translation, appear, trial_d, trial_r, success, weight):
        self.word = word
        self.typ = typ
        self.translation = translation
        self.example = example
        self.example_translation = example_translation
        self.russian = russian
        self.appear = appear
        self.trial_d = trial_d
        self.trial_r = trial_r
        self.success = success
        self.weight = weight
            
    def getTyp(self):
        return self.typ
    
    def getRussian(self):
        return self.russian
    
    def getExample(self):
        return self.example
    
    def changeType(self,new):
        self.typ = new
    
    def getTrials_d(self):
        return self.trial_d
    
    def getTrials_r(self):
        return self.trial_r
    
    def addTrials_d(self):
        self.trial_d += 1
        
    def addTrials_r(self):
        self.trial_r += 1
    
    def getAppear(self):
        return self.appear
    
    def addAppear(self):
        self.appear += 1
        return self.appear
       
    def addSuccess(self):
        self.success += 1
        return self.success   
    
    def getSuccess(self):
        return self.success
    
    def getWord(self):
        return self.word
    
    def getTranslation(self):
        return self.translation
    
    def getWeight(self):
        if self.trial_d == 0 and self.trial_r == 0:
            return self.weight
        else: 
            return 100 - (self.success/ (self.trial_d + self.trial_r)) * 100
        
    
    def __len__(self):
        return len(f'{self.word} -> {self.translation}')
    
    def __repr__(self):
        return f'{self.word}: {self.getTranslation()} , {self.getExample()}'
    
    def __str__(self):
        return f'{self.word} : {self.translation}'
    
    
class Lesson(object):
    def __init__(self, number):
        self.number = number
    def getNumber(self):
        return self.number
    def start(self, start):
        self.start = start
    def getStart(self):
        return self.start
    def inter(self, inter):
        self.inter = inter
    def getInter(self):
        return self.inter
    def finish(self, finish):
        self.finish = finish
    def getFinish(self):
        return self.finish
    def number_of_easy(self,easy):
        self.easy = easy
    def getNumber_of_easy(self):
        return self.easy
    def points(self, pts):
        self.pts = pts
    def add_pts(self, a_pts):
        self.pts += a_pts
    def getPoints(self):
        return self.pts + 1500 - int((self.finish - self.start).seconds)
    def wlist(self,list_of_words):
        self.list_of_words = list_of_words
    def getList(self):
        return self.list_of_words
    def length_of_lesson(self,lol):
        self.lol = lol
    def getLength_of_lesson(self):
        return self.lol

In [60]:
#defs

def loadWords(df_words, temp):
    wordList = []
    for i in range (len(words)):
        if temp == 'yes':
            wordList.append(Words(words.iloc[i,0],words.iloc[i,1],words.iloc[i,2],words.iloc[i,5], words.iloc[i,3],words.iloc[i,4],words.iloc[i,6],words.iloc[i,7],words.iloc[i,8],words.iloc[i,9],words.iloc[i,10]))
        else:
            wordList.append(Words(words.iloc[i,0],words.iloc[i,1],words.iloc[i,2],words.iloc[i,5], words.iloc[i,3],words.iloc[i,4],0,0,0,0, 100))       
    return wordList


def no_space(word):
    lst = []
    for w in word:
        lst.append(w.strip())
    return lst


def translation_with_comma(translation):
    try:
        translation = translation.split(',')
        translation = no_space(translation)
        return translation
    
    except:
        return [translation]
    
def help_for_guess(word, r): #r - number of letters
    list_of_word = list(word)
    copy = list_of_word.copy()
    for j in range(len(list_of_word)):
        if list_of_word[j] == ' ':
            copy.remove(list_of_word[j])
    
    l = random.sample(copy, k = r)
    
    for i in range(len(list_of_word)):
        if list_of_word[i] in l:
            list_of_word[i] = list_of_word[i]
        else:
            list_of_word[i] = '_'      
    back = ' '.join(list_of_word)
    
    return back


def rightword(word,translation,rever=0):
    point_counter = 0
    if rever == 1:
        b = 0
        while b == 0:
            x = input(f'[à ë ï é]\n\nPress "1","2","3" to open 1, 2, 3 letters in the word\n\n {translation}: ')
            if x == word:
                b = 1
                return [True, point_counter]
            elif x == '1' or x == '2' or x == '3':
                print(help_for_guess(word, int(x)))
                point_counter += int(x)
            else:
                return [False, point_counter]
                b = 1
    elif rever == 0:
        b = 0
        translation = translation_with_comma(translation)
        while b == 0:
            x = input(f'[à ë ï é]\n\nPress "1","2","3" to open 1, 2, 3 letters in the word\n\n {word}: ')
            if x in translation:
                b = 1
                return [True, point_counter]
            elif x == '1' or x == '2' or x == '3':
                print(help_for_guess(translation[0], int(x)))
                point_counter += int(x)
            else:
                b = 1
                return [False, point_counter]
    

def the_longest(sample_of_words):
    mx = 0
    for i in sample_of_words:
        if len(i) > mx:
            mx = len(i)
    return mx


def cycle(sample_of_words, rever):
    p = 250 
    s = sample_of_words.copy()
    while len(s) > 0:
        random.shuffle(s)
        lst_to_delete = []
        for i in s:
            temp = rightword(i.getWord(),i.getTranslation(),rever)
            if temp[0]:
                print("It's right!")
                p = p - temp[1]
                print(temp[1], p)
                i.addSuccess()
                if rever == 0:
                    i.addTrials_d()
                else:
                    i.addTrials_r()
                lst_to_delete.append(i)
            else:
                print("Shame on you!")
                p = p - temp[1]
                p -= 1
                print(p)
                if rever == 0:
                    i.addTrials_d()
                else:
                    i.addTrials_r()            
        if len(lst_to_delete) > 0:
            for w in lst_to_delete:
                s.remove(w)
            if len(s) != 0:
                plotting(s)
        else:
            if len(s) != 0:
                plotting(s)
            
            
    return p

def list_to_list(lst):
    st = ''
    for w in lst:
        st = st + w + '; '
    st.strip()       
    st = st[0:(len(st)-2)]
    return st

def lessonLenght(list_of_words):
    lessonLenght = 0
    for w in list_of_words:
        temp = w.getWord()
        temp = temp.replace(' ','')
        lessonLenght += len(temp)
    return lessonLenght

def random_sample(list_of_words, n):
    sample = random.choices(list_of_words, weights = [w.getWeight() for w in list_of_words], k = n )
    l = len(sample)
    st = 0
    count = 0
    while l != st:
        st = len(set(sample))
        sample = random.choices(list_of_words, weights = [w.getWeight() for w in list_of_words], k = n )
        count += 1
    print(count)
    
    return sample

In [117]:
#object creation
exist = 'yes'
wordList = loadWords(words,exist)
sample = random_sample(wordList,25)   

1


In [91]:
for w in sample:
    try:
        math.isnan(w.getExample())
    except:
        print(w.getExample())

Haar cliënt had geen gemakkelijke tijd gehad sinds haar behandeling begon.
De koning is te naïef om de ernst van de situatie in te zien.
Ze gingen in een kring zitten.
Daarna wordt het ter goedkeuring voorgelegd aan de rechters.
Laura deed spontaan een stap naar hem toe.
Wie goed keek, zag dat ik tranen in mijn ogen had.
De ploeg met de beste winnaarsmentaliteit zal zegevieren.
De blauwe oceaan ligt er rustig bij.
Ze werkte door tot haar pensioen.
Met een besliste beweging sloot hij de deur.
Ze bouwen een huis op de grond van zijn vader.
Bij dreiging van oorlog wordt de noodtoestand uitgeroepen.
Bert deelt zijn appel zonder dat hij daartoe wordt gedwongen.


In [118]:
temp = []
for w in sample:
    print(w.getTyp(), w)

for w in sample:
    #print(w.getTyp())
    try:
        math.isnan(w.getTyp())
        temp.append(w)
    except:
        if re.search(r'de', w.getTyp()) and not re.search(r'het', w.getTyp()):
            print(r'de', w)
        elif re.search(r'het', w.getTyp()) and not re.search(r'de', w.getTyp()):
            print('het', w)
        elif re.search(r'het', w.getTyp()) and re.search(r'de', w.getTyp()):
            print('de/het', w)
        else:
            print(w)
for t in temp:
    print(t)

noun, de(m) race : race
noun, pi data : data
adj akelig : nasty
de schouwburg : theater
verb houden : to hold
adj zelfstandig : independent
adj dankbaar : grateful
noun, de(f) situatie : situation
noun, de(f) afwachting : expectation
noun, de(m) race : race
noun, het district : district
adj vrouwelijk : female
verb verheugen (zich) : to be happy
nan Nieuwjaar : New year
verb verdelen : to divide
nan denkbeeldig : imaginary
verb reclame maken : Advertise
noun, de(f) politie : police
het etmaal : day
verb neigen : to incline
noun, de(f) aantekening : note
noun, de supermarkt : supermarket
noun, de hoede : guard
verb samenwerken : to collaborate
nan inslapen : to fall asleep
de race : race
data : data
akelig : nasty
de schouwburg : theater
houden : to hold
zelfstandig : independent
dankbaar : grateful
de situatie : situation
de afwachting : expectation
de race : race
het district : district
vrouwelijk : female
verheugen (zich) : to be happy
verdelen : to divide
reclame maken : Advertise
d

In [79]:
temp

[iedere: every , nan, van harte: congratulations , nan, italië: Italy , nan]

In [74]:
temp

[iedere: every , nan, van harte: congratulations , nan, italië: Italy , nan]

In [85]:
import re
m = re.search('de', 'het')
if re.search('de', 'de'):
    print('de')

de


In [122]:
w = 'football'

if re.search(r'de', 'de') and not re.search(r'het', 'de'):
    print('de', w)
elif re.search(r'het', 'de') and not re.search(r'de', 'de'):
    print('het', w)
elif re.search(r'het','de') and re.search(r'de', 'de'):
    print('de/het', w)    
else:
    print(w)

de football


In [19]:
df = lesson

df = df.assign(inter_pts = lambda x: ((lesson['inter'] - lesson['start'])))
df['points'] = df['inter_pts'].apply(lambda x: x.seconds)

In [20]:
df

,lesson,start,inter,finish,known,points,length,time,list_of_words,r,inter_pts
0,1,2022-11-26 21:14:16.069,2022-11-26 21:24:14.423,2022-11-26 21:31:48.654,6,598,184,1052,krapte; argument; jouw; achterwerk; broedstoof...,1,0 days 00:09:58.354000
1,2,2022-11-27 02:30:31.842,2022-11-27 02:41:47.490,2022-11-27 02:49:34.282,8,675,185,1142,aanranden; fiks; middel; kroeshaar; plezant; a...,2,0 days 00:11:15.648000
2,3,2022-11-27 13:24:02.594,2022-11-27 13:35:42.708,2022-11-27 13:42:12.905,8,700,204,1090,reis; betekenis; naast zitten; gewoon; voortaa...,3,0 days 00:11:40.114000
3,4,2022-11-27 22:23:03.719,2022-11-27 22:34:28.615,2022-11-27 22:45:14.736,7,684,217,1331,recept; opzichtig; bloed; militair; verzuimen;...,4,0 days 00:11:24.896000
4,5,2022-11-28 17:44:21.460,2022-11-28 17:52:57.397,2022-11-28 17:58:23.277,5,515,191,841,Romeins; storten; half; alsnog; zuchten; rein;...,5,0 days 00:08:35.937000
...,...,...,...,...,...,...,...,...,...,...,...
116,117,2023-01-14 13:27:12.907,2023-01-14 13:32:16.720,2023-01-14 13:35:21.312,10,303,187,488,geslaagd; werking; heus; buiten schot; vent; r...,93,0 days 00:05:03.813000
117,118,2023-01-14 13:56:22.231,2023-01-14 14:00:40.230,2023-01-14 14:05:24.144,25,257,220,541,beduidend; pot; niezen; ingesloten; schooljaar...,9,0 days 00:04:17.999000
118,119,2023-01-14 14:06:32.347,2023-01-14 14:12:39.491,2023-01-14 14:19:09.496,25,367,246,757,de stuipen op het lijf jagen; pensioneren; vei...,999,0 days 00:06:07.144000
119,120,2023-01-15 12:23:42.412,2023-01-15 12:30:49.165,2023-01-15 12:35:38.402,8,426,202,715,doei; gemeentebestuur; stemming; doelstelling;...,94,0 days 00:07:06.753000


In [13]:
lesson['inter']

0     2022-11-26 21:24:14.423
1     2022-11-27 02:41:47.490
2     2022-11-27 13:35:42.708
3     2022-11-27 22:34:28.615
4     2022-11-28 17:52:57.397
                ...          
116   2023-01-14 13:32:16.720
117   2023-01-14 14:00:40.230
118   2023-01-14 14:12:39.491
119   2023-01-15 12:30:49.165
120   2023-01-15 12:45:42.046
Name: inter, Length: 121, dtype: datetime64[ns]

In [17]:
lesson3['inter_sec'] = lesson3['inter_pts'].apply(lambda x: x.seconds)


In [18]:
lesson3

,lesson,start,inter,finish,known,points,length,time,list_of_words,r,inter_pts,inter_sec
0,1,2022-11-26 21:14:16.069,2022-11-26 21:24:14.423,2022-11-26 21:31:48.654,6,877,184,1052,krapte; argument; jouw; achterwerk; broedstoof...,1,0 days 00:09:58.354000,598
1,2,2022-11-27 02:30:31.842,2022-11-27 02:41:47.490,2022-11-27 02:49:34.282,8,803,185,1142,aanranden; fiks; middel; kroeshaar; plezant; a...,2,0 days 00:11:15.648000,675
2,3,2022-11-27 13:24:02.594,2022-11-27 13:35:42.708,2022-11-27 13:42:12.905,8,865,204,1090,reis; betekenis; naast zitten; gewoon; voortaa...,3,0 days 00:11:40.114000,700
3,4,2022-11-27 22:23:03.719,2022-11-27 22:34:28.615,2022-11-27 22:45:14.736,7,580,217,1331,recept; opzichtig; bloed; militair; verzuimen;...,4,0 days 00:11:24.896000,684
4,5,2022-11-28 17:44:21.460,2022-11-28 17:52:57.397,2022-11-28 17:58:23.277,5,1115,191,841,Romeins; storten; half; alsnog; zuchten; rein;...,5,0 days 00:08:35.937000,515
...,...,...,...,...,...,...,...,...,...,...,...,...
116,117,2023-01-14 13:27:12.907,2023-01-14 13:32:16.720,2023-01-14 13:35:21.312,10,1497,187,488,geslaagd; werking; heus; buiten schot; vent; r...,93,0 days 00:05:03.813000,303
117,118,2023-01-14 13:56:22.231,2023-01-14 14:00:40.230,2023-01-14 14:05:24.144,25,1425,220,541,beduidend; pot; niezen; ingesloten; schooljaar...,9,0 days 00:04:17.999000,257
118,119,2023-01-14 14:06:32.347,2023-01-14 14:12:39.491,2023-01-14 14:19:09.496,25,1217,246,757,de stuipen op het lijf jagen; pensioneren; vei...,999,0 days 00:06:07.144000,367
119,120,2023-01-15 12:23:42.412,2023-01-15 12:30:49.165,2023-01-15 12:35:38.402,8,1257,202,715,doei; gemeentebestuur; stemming; doelstelling;...,94,0 days 00:07:06.753000,426


In [47]:
df = lesson.copy()
df = df.assign(inter_pts = lambda x: ((lesson['inter'] - lesson['start'])))
df['points'] = df['inter_pts'].apply(lambda x: x.seconds)

In [28]:
mod_lesson = df.sort_values(by='points', ascending=True, ignore_index=True)

In [32]:
mod_lesson[mod_lesson['known'] != 25]

,lesson,start,inter,finish,known,points,length,time,list_of_words,r,inter_pts
12,53,2022-12-17 13:02:38.020,2022-12-17 13:07:33.567,2022-12-17 13:12:56.103,15,295,191,618,plukken; klantenservice; uitgang; er vanuit ga...,42,0 days 00:04:55.547000
13,76,2022-12-27 13:05:34.662,2022-12-27 13:10:32.008,2022-12-27 13:13:42.698,15,297,208,488,fantasie; vis; erbarmelijk; tentoonstelling; m...,60,0 days 00:04:57.346000
15,117,2023-01-14 13:27:12.907,2023-01-14 13:32:16.720,2023-01-14 13:35:21.312,10,303,187,488,geslaagd; werking; heus; buiten schot; vent; r...,93,0 days 00:05:03.813000
16,115,2023-01-13 17:47:59.944,2023-01-13 17:53:03.137,2023-01-13 17:56:57.061,11,303,185,537,draaglijk; cyclus; nogmaals; ontwarren; dronke...,91,0 days 00:05:03.193000
17,109,2023-01-08 17:00:15.625,2023-01-08 17:05:19.910,2023-01-08 17:09:13.948,11,304,182,538,insgelijks; visie; van nature; kop; manipulere...,85,0 days 00:05:04.285000
...,...,...,...,...,...,...,...,...,...,...,...
117,2,2022-11-27 02:30:31.842,2022-11-27 02:41:47.490,2022-11-27 02:49:34.282,8,675,185,1142,aanranden; fiks; middel; kroeshaar; plezant; a...,2,0 days 00:11:15.648000
118,4,2022-11-27 22:23:03.719,2022-11-27 22:34:28.615,2022-11-27 22:45:14.736,7,684,217,1331,recept; opzichtig; bloed; militair; verzuimen;...,4,0 days 00:11:24.896000
119,12,2022-12-01 19:47:34.473,2022-12-01 19:59:11.655,2022-12-01 20:07:38.151,4,697,205,1203,geraken; ambtenaar; zwemmen; schepsel; verhoge...,12,0 days 00:11:37.182000
120,3,2022-11-27 13:24:02.594,2022-11-27 13:35:42.708,2022-11-27 13:42:12.905,8,700,204,1090,reis; betekenis; naast zitten; gewoon; voortaa...,3,0 days 00:11:40.114000


In [44]:
df.tail(5)

,lesson,start,inter,finish,known,points,length,time,list_of_words,r,inter_pts
117,118,2023-01-14 13:56:22.231,2023-01-14 14:00:40.230,2023-01-14 14:05:24.144,25,257,220,541,beduidend; pot; niezen; ingesloten; schooljaar...,9,0 days 00:04:17.999000
118,119,2023-01-14 14:06:32.347,2023-01-14 14:12:39.491,2023-01-14 14:19:09.496,25,367,246,757,de stuipen op het lijf jagen; pensioneren; vei...,999,0 days 00:06:07.144000
119,120,2023-01-15 12:23:42.412,2023-01-15 12:30:49.165,2023-01-15 12:35:38.402,8,426,202,715,doei; gemeentebestuur; stemming; doelstelling;...,94,0 days 00:07:06.753000
120,121,2023-01-15 12:40:26.285,2023-01-15 12:45:42.046,2023-01-15 12:51:57.535,10,315,237,691,lak aan wat (iemand zegt/doet) hebben; plat; g...,95,0 days 00:05:15.761000
121,122,2023-01-15 18:15:55.744,2023-01-15 18:21:18.599,2023-01-15 18:26:48.058,14,500,206,652,aan een stuk door; zwaaien; doen; opvoeden; ve...,96,0 days 00:05:22.855000


In [58]:
row = df.index.values[-1]

df.loc[row, 'lesson'] = 95
df.loc[row, 'start'] = 0
df.loc[row, 'inter'] = 0
df.loc[row, 'finish'] = 0
df.loc[row, 'known'] = 0
df.loc[row, 'points'] = 240
df.loc[row, 'length'] = 0
df.loc[row, 'time'] = 0
df.loc[row, 'list_of_words'] = 0
df.loc[row, 'r'] = 0

In [59]:
df

,lesson,start,inter,finish,known,points,length,time,list_of_words,r,inter_pts
0,1,2022-11-26 21:14:16.069000,2022-11-26 21:24:14.423000,2022-11-26 21:31:48.654000,6,598,184,1052,krapte; argument; jouw; achterwerk; broedstoof...,1,0 days 00:09:58.354000
1,2,2022-11-27 02:30:31.842000,2022-11-27 02:41:47.490000,2022-11-27 02:49:34.282000,8,675,185,1142,aanranden; fiks; middel; kroeshaar; plezant; a...,2,0 days 00:11:15.648000
2,3,2022-11-27 13:24:02.594000,2022-11-27 13:35:42.708000,2022-11-27 13:42:12.905000,8,700,204,1090,reis; betekenis; naast zitten; gewoon; voortaa...,3,0 days 00:11:40.114000
3,4,2022-11-27 22:23:03.719000,2022-11-27 22:34:28.615000,2022-11-27 22:45:14.736000,7,684,217,1331,recept; opzichtig; bloed; militair; verzuimen;...,4,0 days 00:11:24.896000
4,5,2022-11-28 17:44:21.460000,2022-11-28 17:52:57.397000,2022-11-28 17:58:23.277000,5,515,191,841,Romeins; storten; half; alsnog; zuchten; rein;...,5,0 days 00:08:35.937000
...,...,...,...,...,...,...,...,...,...,...,...
117,118,2023-01-14 13:56:22.231000,2023-01-14 14:00:40.230000,2023-01-14 14:05:24.144000,25,257,220,541,beduidend; pot; niezen; ingesloten; schooljaar...,9,0 days 00:04:17.999000
118,119,2023-01-14 14:06:32.347000,2023-01-14 14:12:39.491000,2023-01-14 14:19:09.496000,25,367,246,757,de stuipen op het lijf jagen; pensioneren; vei...,999,0 days 00:06:07.144000
119,120,2023-01-15 12:23:42.412000,2023-01-15 12:30:49.165000,2023-01-15 12:35:38.402000,8,426,202,715,doei; gemeentebestuur; stemming; doelstelling;...,94,0 days 00:07:06.753000
120,121,2023-01-15 12:40:26.285000,2023-01-15 12:45:42.046000,2023-01-15 12:51:57.535000,10,315,237,691,lak aan wat (iemand zegt/doet) hebben; plat; g...,95,0 days 00:05:15.761000


In [60]:
df.loc[:, "points"][-1:].values[0]

240

In [62]:
xd = df.sort_values(by='points', ascending=True, ignore_index=True)
pl = xd[xd['points'] == 240].index[0]

In [63]:
pl

5

In [66]:
xd.head(6) 

,lesson,start,inter,finish,known,points,length,time,list_of_words,r,inter_pts
0,39,2022-12-12 20:36:30.669000,2022-12-12 20:39:42.275000,2022-12-12 20:43:24.561000,25,191,205,413,Romeins; storten; half; alsnog; zuchten; rein;...,5,0 days 00:03:11.606000
1,27,2022-12-06 23:10:05.756000,2022-12-06 23:13:39.910000,2022-12-06 23:18:01.098000,25,214,204,475,reis; betekenis; naast zitten; gewoon; voortaa...,3,0 days 00:03:34.154000
2,86,2022-12-29 13:22:10.292000,2022-12-29 13:25:49.796000,2022-12-29 13:30:28.277000,25,219,191,497,veilen; eenvoudig; bezwaarlijk; taxi; politieb...,46,0 days 00:03:39.504000
3,19,2022-12-04 21:40:19.907000,2022-12-04 21:44:09.949000,2022-12-04 21:48:35.453000,25,230,184,495,krapte; argument; jouw; achterwerk; broedstoof...,1,0 days 00:03:50.042000
4,66,2022-12-23 22:17:43.081000,2022-12-23 22:21:37.034000,2022-12-23 22:25:12.082000,25,233,185,449,bruikbaar; gekruid; erom; tiental; nieuwsgieri...,19,0 days 00:03:53.953000
5,95,0,0,0,0,240,0,0,0,0,0 days 00:05:22.855000
